In [1]:
import yfinance as yf
import pandas as pd
from IPython.display import display
from scipy.stats import zscore
import numpy as np
import torch
import gym
from gym import spaces
from envs.portfolio_env import Mult_Asset_portEnv
from envs.portfolio_env_dqn import Mult_Asset_portEnv_DQN
import random

from stable_baselines3 import A2C, DQN, PPO
from stable_baselines3.common.vec_env import DummyVecEnv

2025-07-21 20:33:35.253040: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1753126416.438450   57106 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1753126416.734617   57106 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-07-21 20:33:39.505477: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
assets = ['NVDA', 'AAPL', 'AMZN', 'JPM', 'IBM', 'MSFT', 'TSLA', 'GOOGL', 'META', 'HSBC']
start_date ='2019-01-01'
end_date = '2025-01-01'
 
df = yf.download(tickers=assets, start= start_date , end= end_date, interval='1d')

df 

/tmp/ipykernel_57106/1496721472.py:5: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(tickers=assets, start= start_date , end= end_date, interval='1d')
[*********************100%***********************]  10 of 10 completed


Price            Close                                                 \
Ticker            AAPL        AMZN       GOOGL       HSBC         IBM   
Date                                                                    
2019-01-02   37.617847   76.956497   52.419624  27.954206   82.193146   
2019-01-03   33.870838   75.014000   50.967827  27.646484   80.552269   
2019-01-04   35.316761   78.769501   53.582146  28.453379   83.698471   
2019-01-07   35.238155   81.475502   53.475288  28.070444   84.290588   
2019-01-08   35.909893   82.829002   53.944973  28.145668   85.489151   
...                ...         ...         ...        ...         ...   
2024-12-24  257.578674  229.050003  195.647552  47.340084  221.451202   
2024-12-26  258.396667  227.050003  195.138763  47.349697  221.924866   
2024-12-27  254.974930  223.750000  192.305450  47.378536  219.842682   
2024-12-30  251.593094  221.300003  190.789032  47.590000  217.346039   
2024-12-31  249.817383  219.389999  188.853622  47.541943  216.931580   

Price                                                                   ...  \
Ticker             JPM        META        MSFT        NVDA        TSLA  ...   
Date                                                                    ...   
2019-01-02   82.013519  134.939713   94.945518    3.377545   20.674667  ...   
2019-01-03   80.847954  131.021210   91.452652    3.173484   20.024000  ...   
2019-01-04   83.828438  137.197327   95.706055    3.376801   21.179333  ...   
2019-01-07   83.886726  137.296783   95.828110    3.555572   22.330667  ...   
2019-01-08   83.728523  141.752335   96.522934    3.467054   22.356667  ...   
...                ...         ...         ...         ...         ...  ...   
2024-12-24  238.440521  606.742920  437.647369  140.197372  462.279999  ...   
2024-12-26  239.257263  602.350220  436.432068  139.907410  454.130005  ...   
2024-12-27  237.318726  598.816101  428.881104  136.987869  431.660004  ...   
2024-12-30  235.498260  590.260254  423.202911  137.467804  417.410004  ...   
2024-12-31  235.882034  584.539795  419.885681  134.268326  403.839996  ...   

Price          Volume                                                   \
Ticker           AAPL       AMZN     GOOGL     HSBC      IBM       JPM   
Date                                                                     
2019-01-02  148158800  159662000  31868000  2565000  4434935  15670900   
2019-01-03  365248800  139512000  41960000  1479400  4546648  16286400   
2019-01-04  234428400  183652000  46022000  3060900  4683779  16935200   
2019-01-07  219111200  159864000  47446000  2212300  3923755  15430700   
2019-01-08  164101200  177628000  35414000  2888300  4982726  13578800   
...               ...        ...       ...      ...      ...       ...   
2024-12-24   23234700   15007500  10403300   309000  1186200   3729100   
2024-12-26   27237100   16146700  12046600   516300  3286500   4451800   
2024-12-27   42355300   27367100  18891400  1088100  1810800   5730200   
2024-12-30   35557500   28321200  14264700  1344500  2095600   5723800   
2024-12-31   39480700   24819700  17466900   768200  2270200   4871000   

Price                                                 
Ticker          META      MSFT       NVDA       TSLA  
Date                                                  
2019-01-02  28146200  35329300  508752000  174879000  
2019-01-03  22717900  42579100  705552000  104478000  
2019-01-04  29002100  44060600  585620000  110911500  
2019-01-07  20089300  35656100  709160000  113268000  
2019-01-08  26263800  31514400  786016000  105127500  
...              ...       ...        ...        ...  
2024-12-24   4726100   7164500  105157000   59551800  
2024-12-26   6081400   8194200  116205600   76366400  
2024-12-27   8084200  18117700  170582600   82666800  
2024-12-30   7025900  13158700  167734700   64941000  
2024-12-31   6019500  13246500  155659200   76825100  

[1510 rows x 50 columns]

In [3]:
sta = df.stack()

df_flt = sta.reset_index()
is_multi_index = isinstance(df_flt.index, pd.MultiIndex)  
is_multi_col= isinstance(df_flt.columns, pd.MultiIndex)

print (df_flt.head())
print (df_flt.info())
print (df_flt.describe())
print (df_flt.dtypes)
print (df_flt.isna().sum())
print(df_flt.index.to_series().min())
print(df_flt.index.to_series().max())
print ('Row', is_multi_index)
print ('col', is_multi_col)

/tmp/ipykernel_57106/3544417225.py:1: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  sta = df.stack()


Price       Date Ticker      Close       High        Low       Open     Volume
0     2019-01-02   AAPL  37.617847  37.839383  36.738858  36.896077  148158800
1     2019-01-02   AMZN  76.956497  77.667999  73.046501  73.260002  159662000
2     2019-01-02  GOOGL  52.419624  52.723303  50.958387  51.053815   31868000
3     2019-01-02   HSBC  27.954206  27.974720  27.523405  27.605461    2565000
4     2019-01-02    IBM  82.193146  82.742479  79.681910  79.910201    4434935
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15100 entries, 0 to 15099
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   Date    15100 non-null  datetime64[ns]
 1   Ticker  15100 non-null  object        
 2   Close   15100 non-null  float64       
 3   High    15100 non-null  float64       
 4   Low     15100 non-null  float64       
 5   Open    15100 non-null  float64       
 6   Volume  15100 non-null  int64         
dtypes: datetime64[ns]

In [4]:
# for symbol in assets:
#     df = yf.download(tickers=symbol, start= start_date , end= end_date, interval='1d')
#     sta = df.stack()
#     df_flt = sta.reset_index()
#     df_flt.to_csv(f'/home/micheal/Documents/Python_Library/RL_Optimization_Portfolio/data/raw/{symbol}_daily.CSV')


In [5]:

appl = pd.read_csv('/home/micheal/Documents/Python_Library/RL_Optimization_Portfolio/data/raw/AAPL_daily.CSV', parse_dates=["Date"], index_col="Date")
msft = pd.read_csv('/home/micheal/Documents/Python_Library/RL_Optimization_Portfolio/data/raw/MSFT_daily.CSV', parse_dates=["Date"], index_col="Date")


df = pd.concat([appl, msft])


is_multi_index = isinstance(df.index, pd.MultiIndex)  
is_multi_col= isinstance(df.columns, pd.MultiIndex)

df = df.drop(columns=['Unnamed: 0']).copy()

print (df.head())
print (df.columns)
print (df[['Ticker']])
print (df.info())
print (df.describe())
print (df.dtypes)
print (df.isna().sum())
print (df.index.to_series().min())
print (df.index.to_series().max())
print ('Row', is_multi_index)
print ('col', is_multi_col)

           Ticker      Close       High        Low       Open     Volume
Date                                                                    
2019-01-02   AAPL  37.617844  37.839379  36.738855  36.896073  148158800
2019-01-03   AAPL  33.870834  34.711709  33.825574  34.297226  365248800
2019-01-04   AAPL  35.316757  35.385840  34.254350  34.428241  234428400
2019-01-07   AAPL  35.238152  35.452541  34.754589  35.421573  219111200
2019-01-08   AAPL  35.909897  36.164781  35.378693  35.626428  164101200
Index(['Ticker', 'Close', 'High', 'Low', 'Open', 'Volume'], dtype='object')
           Ticker
Date             
2019-01-02   AAPL
2019-01-03   AAPL
2019-01-04   AAPL
2019-01-07   AAPL
2019-01-08   AAPL
...           ...
2024-12-24   MSFT
2024-12-26   MSFT
2024-12-27   MSFT
2024-12-30   MSFT
2024-12-31   MSFT

[3020 rows x 1 columns]
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 3020 entries, 2019-01-02 to 2024-12-31
Data columns (total 6 columns):
 #   Column  Non-Null Count  D

In [6]:
df['LogReturn'] = df.groupby('Ticker')['Close'].transform(lambda x: np.log(x / x.shift(1)))

df.dropna(subset=['LogReturn'], inplace=True)

df['LogReturn_Z'] = df.groupby('Ticker')['LogReturn'].transform(lambda x: (x - x.mean())/ x.std())

print (df.head())
print (df.isna().sum())
print (df[df.isna().any(axis=1)])
print (df.tail())


           Ticker      Close       High        Low       Open     Volume  \
Date                                                                       
2019-01-03   AAPL  33.870834  34.711709  33.825574  34.297226  365248800   
2019-01-04   AAPL  35.316757  35.385840  34.254350  34.428241  234428400   
2019-01-07   AAPL  35.238152  35.452541  34.754589  35.421573  219111200   
2019-01-08   AAPL  35.909897  36.164781  35.378693  35.626428  164101200   
2019-01-09   AAPL  36.519714  36.810329  35.643110  36.038533  180396400   

            LogReturn  LogReturn_Z  
Date                                
2019-01-03  -0.104924    -5.465055  
2019-01-04   0.041803     2.087049  
2019-01-07  -0.002228    -0.179263  
2019-01-08   0.018884     0.907367  
2019-01-09   0.016839     0.802147  
Ticker         0
Close          0
High           0
Low            0
Open           0
Volume         0
LogReturn      0
LogReturn_Z    0
dtype: int64
Empty DataFrame
Columns: [Ticker, Close, High, Low, Open, V

In [7]:

# df.to_csv('/home/micheal/Documents/Python_Library/RL_Optimization_Portfolio/data/processed/AP_MS_daily.CSV')


In [8]:
addres = '/home/micheal/Documents/Python_Library/RL_Optimization_Portfolio/data/processed/AP_MS_daily.CSV'
raw = pd.read_csv(addres, parse_dates=['Date'], index_col='Date', engine='pyarrow', dtype_backend='pyarrow')

def shrinking_ints (df):
    mapping = {}
    for col in df.dtypes [df.dtypes=='int64[pyarrow]'].index:
        max_ = df[col].max()
        min_ = df[col].min()
        if min_ < 0:
            continue
        elif max_ < 255:
            mapping[col] = 'uint8[pyarrow]'
        elif max_ <65_535:
            mapping[col] = 'uint16[pyarrow]'
        elif max_ < 4294967295:
            mapping[col] = 'uint32[pyarrow]'
    return df.astype(mapping)

def clean(df):
    return (df
    .assign(**df.select_dtypes('string').replace('', 'Missing').astype('category'))
    .pipe(shrinking_ints)
    )
    
df = clean(raw)



In [9]:
feat_to_pivot = ['Close', 'High', 'Low', 'Open', 'Volume', 'LogReturn', 'LogReturn_Z']

pivoted_featu = []

for feature in feat_to_pivot:
    feat_pivot = df.pivot_table(index='Date', columns='Ticker', values = feature)
    feat_pivot.columns = [f"{ticker}_{feature}" for ticker in feat_pivot.columns]
    pivoted_featu.append(feat_pivot)
    
df_wide = pd.concat(pivoted_featu, axis=1)

org_rows = len(df_wide)
df_wide.dropna(inplace=True)
rows_after_drop = len(df_wide)
if org_rows != rows_after_drop:
    print(1)
    
df_wide = df_wide.sort_index(axis=1)

print (df_wide.head())
print (df_wide.info())
print (df_wide.columns)

/tmp/ipykernel_57106/607006344.py:6: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  feat_pivot = df.pivot_table(index='Date', columns='Ticker', values = feature)
/tmp/ipykernel_57106/607006344.py:6: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  feat_pivot = df.pivot_table(index='Date', columns='Ticker', values = feature)
/tmp/ipykernel_57106/607006344.py:6: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  feat_pivot = df.pivot_table(index='Date', columns='Ticker', values = feature)
/tmp/ipykernel_57106/6070

            AAPL_Close  AAPL_High  AAPL_LogReturn  AAPL_LogReturn_Z  \
Date                                                                  
2019-01-03   33.870834  34.711709       -0.104924         -5.465055   
2019-01-04   35.316757   35.38584        0.041803          2.087049   
2019-01-07   35.238152  35.452541       -0.002228         -0.179263   
2019-01-08   35.909897  36.164781        0.018884          0.907367   
2019-01-09   36.519714  36.810329        0.016839          0.802147   

             AAPL_Low  AAPL_Open  AAPL_Volume  MSFT_Close  MSFT_High  \
Date                                                                   
2019-01-03  33.825574  34.297226  365248800.0   91.452667  94.072308   
2019-01-04   34.25435  34.428241  234428400.0   95.706047  96.250633   
2019-01-07  34.754589  35.421573  219111200.0    95.82811  96.964225   
2019-01-08  35.378693  35.626428  164101200.0   96.522919  97.621475   
2019-01-09   35.64311  36.038533  180396400.0   97.903183  98.475936  

In [10]:
# output = "/home/micheal/Documents/Python_Library/RL_Optimization_Portfolio/data/processed/AP_MS_WIDE_daily.CSV"
# df_wide.to_csv(output)

In [11]:
addres = '/home/micheal/Documents/Python_Library/RL_Optimization_Portfolio/data/processed/AP_MS_WIDE_daily.CSV'
df = pd.read_csv(addres, parse_dates=['Date'], index_col='Date', engine='pyarrow', dtype_backend='pyarrow')


try:
    df = pd.read_csv(addres, parse_dates=['Date'], index_col='Date', engine='pyarrow', dtype_backend='pyarrow')

    for col in df.columns:
        if pd.api.types.is_numeric_dtype(df[col]):
            df[col] = df[col].astype(np.float32)


except FileNotFoundError:
    print(f"Error: The wide-format data file '{addres}' was not found.")
    print("Please ensure you have run the data processing steps to create this file correctly.")
    exit() 

features_list = ['Close', 'High', 'Low', 'Open', 'Volume', 'LogReturn', 'LogReturn_Z']

# Dynamically determine the number of assets from the columns of df_wide
all_tickers = sorted(list(set([col.split('_')[0] for col in df.columns if '_' in col])))
num_assets = len(all_tickers)

# --- Environment Initialization ---
print("--- Initializing Multi-Asset Portfolio Environment for A2C Training ---")


--- Initializing Multi-Asset Portfolio Environment for A2C Training ---


In [12]:
env = Mult_Asset_portEnv(df=df,
                         num_assets=num_assets,
                         features_list=features_list,
                         window_size=5)

# 5. Run the simulation
state = env.reset()

print("\n--- Starting Multi-Asset Environment Test ---")

for _ in range(10):
    action = env.action_space.sample()
    action = action / np.sum(action)
    next_step, reward, done, info = env.step(action)
    env.render()
    
    if done:
        print('Episode finished.')
        break

DEBUG: Observation dtype at step 5: float32
DEBUG: Observation dtype at step 5: float32

--- Starting Multi-Asset Environment Test ---
DEBUG: Observation dtype at step 6: float32
Step 6, Total Portfolio Value: 999.27, Cash: 270.03, Weights: ['0.38', '0.35', '0.27']
DEBUG: Observation dtype at step 7: float32
Step 7, Total Portfolio Value: 992.76, Cash: 185.45, Weights: ['0.38', '0.43', '0.19']
DEBUG: Observation dtype at step 8: float32
Step 8, Total Portfolio Value: 983.40, Cash: 51.32, Weights: ['0.17', '0.78', '0.05']
DEBUG: Observation dtype at step 9: float32
Step 9, Total Portfolio Value: 1008.40, Cash: 337.42, Weights: ['0.34', '0.33', '0.34']
DEBUG: Observation dtype at step 10: float32
Step 10, Total Portfolio Value: 1013.22, Cash: 257.73, Weights: ['0.62', '0.12', '0.25']
DEBUG: Observation dtype at step 11: float32
Step 11, Total Portfolio Value: 1017.40, Cash: 333.83, Weights: ['0.37', '0.30', '0.33']
DEBUG: Observation dtype at step 12: float32
Step 12, Total Portfolio Val

In [13]:

env = DummyVecEnv([lambda: Mult_Asset_portEnv(df=df,
                                             num_assets=num_assets,
                                             features_list=features_list,
                                             window_size=5)])

# --- A2C Model Initialization and Training ---
print("\n--- Initializing A2C Model and Starting Training ---")
model_A2C = A2C('MlpPolicy', env, verbose=1)

model_A2C.learn(total_timesteps=10000)

print("\n--- A2C Model Training Finished ---")
print("You can now evaluate the trained model_A2C or save it for later use.")

DEBUG: Observation dtype at step 5: float32



--- Initializing A2C Model and Starting Training ---
Using cpu device


/home/micheal/anaconda3/lib/python3.12/site-packages/stable_baselines3/common/vec_env/patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


DEBUG: Observation dtype at step 5: float32
DEBUG: Observation dtype at step 6: float32
DEBUG: Observation dtype at step 6: float32
DEBUG: Observation dtype at step 5: float32
DEBUG: Observation dtype at step 5: float32
DEBUG: Observation dtype at step 5: float32
DEBUG: Observation dtype at step 6: float32
DEBUG: Observation dtype at step 7: float32
DEBUG: Observation dtype at step 8: float32
DEBUG: Observation dtype at step 9: float32
DEBUG: Observation dtype at step 10: float32
DEBUG: Observation dtype at step 11: float32
DEBUG: Observation dtype at step 12: float32
DEBUG: Observation dtype at step 12: float32
DEBUG: Observation dtype at step 5: float32
DEBUG: Observation dtype at step 6: float32
DEBUG: Observation dtype at step 7: float32
DEBUG: Observation dtype at step 8: float32
DEBUG: Observation dtype at step 9: float32
DEBUG: Observation dtype at step 10: float32
DEBUG: Observation dtype at step 11: float32
DEBUG: Observation dtype at step 12: float32
DEBUG: Observation dtype 

In [14]:
# model_A2C.save("/home/micheal/Documents/Python_Library/RL_Optimization_Portfolio/models/a2c_model")

In [15]:
print ('\n--- Initializing PPO Model and Starting Training')

model_ppo = PPO('MlpPolicy', env, verbose=1)

model_ppo.learn(total_timesteps=1000)

print("\n--- PPO Model Training Finished ---")
print("You can now evaluate the trained PPO model or save it for later use.")


--- Initializing PPO Model and Starting Training
Using cpu device


DEBUG: Observation dtype at step 5: float32
DEBUG: Observation dtype at step 6: float32
DEBUG: Observation dtype at step 7: float32
DEBUG: Observation dtype at step 8: float32
DEBUG: Observation dtype at step 8: float32
DEBUG: Observation dtype at step 5: float32
DEBUG: Observation dtype at step 6: float32
DEBUG: Observation dtype at step 7: float32
DEBUG: Observation dtype at step 8: float32
DEBUG: Observation dtype at step 9: float32
DEBUG: Observation dtype at step 10: float32
DEBUG: Observation dtype at step 10: float32
DEBUG: Observation dtype at step 5: float32
DEBUG: Observation dtype at step 6: float32
DEBUG: Observation dtype at step 7: float32
DEBUG: Observation dtype at step 7: float32
DEBUG: Observation dtype at step 5: float32
DEBUG: Observation dtype at step 5: float32
DEBUG: Observation dtype at step 5: float32
DEBUG: Observation dtype at step 5: float32
DEBUG: Observation dtype at step 5: float32
DEBUG: Observation dtype at step 6: float32
DEBUG: Observation dtype at st

In [16]:
# model_ppo.save("/home/micheal/Documents/Python_Library/RL_Optimization_Portfolio/models/ppo_model")

In [17]:
# num_discrete_levels = 5 # As we discussed: 0-4 for each asset/cash dimension

# # Initialize your new discrete environment
# env_dqn = DummyVecEnv([lambda: Mult_Asset_portEnv_DQN(df=df,
#                                                       num_assets=num_assets,
#                                                       features_list=features_list,
#                                                       window_size=5)])

# # --- DQN Model Initialization and Training ---
# print("\n--- Initializing DQN Model and Starting Training ---")
# # 'MlpPolicy' is typically used for DQN in stable_baselines3
# model_dqn = DQN('MlpPolicy', env_dqn, verbose=1)

# # Start training for a specified number of timesteps
# # DQN often requires more timesteps than policy-gradient methods to stabilize
# model_dqn.learn(total_timesteps=20000) # Increased timesteps for DQN example

# print("\n--- DQN Model Training Finished ---")
# print("You can now evaluate the trained DQN model or save it for later use.")

In [19]:
A2C_MODEL_PATH = "/home/micheal/Documents/Python_Library/RL_Optimization_Portfolio/models/a2c_model"
PPO_MODEL_PATH = "/home/micheal/Documents/Python_Library/RL_Optimization_Portfolio/models/ppo_model"



def simulate_agent_performance(model, env_class, df, agent_name, num_assets, features_list, window_size = 5): # num_discrete_levels removed
    print(f"\n--- Simulating {agent_name} historical performance ---")
    
    # Create a temporary environment instance for simulation
    # Only Mult_Asset_portEnv is assumed now
    env = env_class(df=df.copy(), num_assets=num_assets, features_list=features_list, window_size=window_size)
    
    obs = env.reset() # Reset the temporary environment
    
    daily_returns = []
    
    # Iterate through the historical data starting from where the env provides observations
    for i in range(env.current_step, len(env.df)):
        # Get action from the trained model's policy
        action, _states = model.predict(obs, deterministic=True)
        
        # Take a step in the environment with the predicted action
        obs, reward, done, info = env.step(action)
        
        # Store the daily return (reward + risk_penalty, to get the 'raw' daily return)
        risk_penalty_val = getattr(env, 'risk_penalty', 0.0001) 
        daily_returns.append(reward + risk_penalty_val) 

        if done:
            print(f"Episode finished for {agent_name} simulation at step {i+1}. Reason: {info.get('reason', 'unknown')}")
            break
            
    # Pad daily_returns with NaNs for the initial window_size steps
    padded_daily_returns = [np.nan] * env.window_size + daily_returns 
    
    # Ensure the length matches the original df (in case episode ended early)
    if len(padded_daily_returns) < len(df):
        padded_daily_returns.extend([np.nan] * (len(df) - len(padded_daily_returns)))
        
    print(f"Finished simulating {agent_name}. Captured {len(daily_returns)} daily returns.")
    return pd.Series(padded_daily_returns, index=df.index, name=f"{agent_name}_DailyReturn")


# --- Load Trained Models ---
print("--- Loading Trained Sub-Agent Models (A2C, PPO only) ---")
try:
    model_a2c = A2C.load(A2C_MODEL_PATH)
    model_ppo = PPO.load(PPO_MODEL_PATH)
except Exception as e:
    print(f"Error loading models. Please ensure paths are correct and models are saved: {e}")
    exit()

# --- Simulate and add performance to df_wide ---
df_wide_with_performance = df_wide.copy()

# Simulate A2C performance
a2c_returns_series = simulate_agent_performance(model_a2c, Mult_Asset_portEnv, df_wide, "A2C", num_assets, features_list, window_size=5)
df_wide_with_performance = pd.concat([df_wide_with_performance, a2c_returns_series], axis=1)

# Simulate PPO performance
ppo_returns_series = simulate_agent_performance(model_ppo, Mult_Asset_portEnv, df_wide, "PPO", num_assets, features_list, window_size=5)
df_wide_with_performance = pd.concat([df_wide_with_performance, ppo_returns_series], axis=1)

# --- Save the updated df_wide with performance data ---
# Renamed output file to reflect A2C and PPO only
output_path_perf = '/home/micheal/Documents/Python_Library/RL_Optimization_Portfolio/data/processed/AP_MS_WIDE_with_A2C_PPO_Perf_daily.CSV'
df_wide_with_performance.to_csv(output_path_perf)
print(f"\nUpdated df_wide with A2C and PPO sub-agent performance saved to: {output_path_perf}")

print("\n--- Head of df_wide_with_performance (new columns) ---")
print(df_wide_with_performance.tail()) 
print(df_wide_with_performance.info())

--- Loading Trained Sub-Agent Models (A2C, PPO only) ---

--- Simulating A2C historical performance ---
DEBUG: Observation dtype at step 5: float32
DEBUG: Observation dtype at step 5: float32
DEBUG: Observation dtype at step 6: float32
DEBUG: Observation dtype at step 7: float32
DEBUG: Observation dtype at step 8: float32
DEBUG: Observation dtype at step 9: float32
DEBUG: Observation dtype at step 10: float32
DEBUG: Observation dtype at step 11: float32
DEBUG: Observation dtype at step 12: float32
DEBUG: Observation dtype at step 13: float32
DEBUG: Observation dtype at step 14: float32
DEBUG: Observation dtype at step 15: float32
DEBUG: Observation dtype at step 16: float32
DEBUG: Observation dtype at step 17: float32
DEBUG: Observation dtype at step 18: float32
DEBUG: Observation dtype at step 19: float32
DEBUG: Observation dtype at step 20: float32
DEBUG: Observation dtype at step 21: float32
DEBUG: Observation dtype at step 22: float32
DEBUG: Observation dtype at step 23: float32
DE

In [20]:
!pip install stable-baselines3

In [21]:
import pandas as pd
import numpy as np
from stable_baselines3 import A2C, PPO

# --- IMPORTANT: ADJUST THESE PATHS TO YOUR SAVED MODELS ---
# These are example paths. Make sure they match where you saved your models.
A2C_MODEL_PATH = "/home/micheal/Documents/Python_Library/RL_Optimization_Portfolio/models/a2c_model"
PPO_MODEL_PATH = "/home/micheal/Documents/Python_Library/RL_Optimization_Portfolio/models/ppo_model"

# --- Load the wide-format data ---
data_file_path = '/home/micheal/Documents/Python_Library/RL_Optimization_Portfolio/data/processed/AP_MS_WIDE_daily.CSV'
df_wide = pd.read_csv(data_file_path, parse_dates=['Date'], index_col='Date', engine='pyarrow', dtype_backend='pyarrow')
# Defensive casting to float32 for all numeric columns
for col in df_wide.columns:
    if pd.api.types.is_numeric_dtype(df_wide[col]):
        df_wide[col] = df_wide[col].astype(np.float32)

# Define common environment parameters
features_list = ['Close', 'High', 'Low', 'Open', 'Volume', 'LogReturn', 'LogReturn_Z']
all_tickers = sorted(list(set([col.split('_')[0] for col in df_wide.columns if '_' in col])))
num_assets = len(all_tickers)
window_size = 5 # Standard window size for sub-agents (used for their observations)

# --- Import sub-environment classes ---
# Mult_Asset_portEnv_DQN is no longer needed as DQN is excluded from meta-agent selection.
from envs.portfolio_env import Mult_Asset_portEnv # For A2C, PPO (continuous)

# --- Create a function to simulate an agent's historical performance ---
def simulate_agent_performance(model, env_class, df, agent_name, num_assets, features_list, window_size): # num_discrete_levels removed
    print(f"\n--- Simulating {agent_name} historical performance ---")
    
    # Create a temporary environment instance for simulation
    # Only Mult_Asset_portEnv is assumed now
    env = env_class(df=df.copy(), num_assets=num_assets, features_list=features_list, window_size=window_size)
    
    obs = env.reset() # Reset the temporary environment
    
    daily_returns = []
    
    # Iterate through the historical data starting from where the env provides observations
    for i in range(env.current_step, len(env.df)):
        # Get action from the trained model's policy
        action, _states = model.predict(obs, deterministic=True)
        
        # Take a step in the environment with the predicted action
        obs, reward, done, info = env.step(action)
        
        # Store the daily return (reward + risk_penalty, to get the 'raw' daily return)
        risk_penalty_val = getattr(env, 'risk_penalty', 0.0001) 
        daily_returns.append(reward + risk_penalty_val) 

        if done:
            print(f"Episode finished for {agent_name} simulation at step {i+1}. Reason: {info.get('reason', 'unknown')}")
            break
            
    # Pad daily_returns with NaNs for the initial window_size steps
    padded_daily_returns = [np.nan] * env.window_size + daily_returns 
    
    # Ensure the length matches the original df (in case episode ended early)
    if len(padded_daily_returns) < len(df):
        padded_daily_returns.extend([np.nan] * (len(df) - len(padded_daily_returns)))
        
    print(f"Finished simulating {agent_name}. Captured {len(daily_returns)} daily returns.")
    return pd.Series(padded_daily_returns, index=df.index, name=f"{agent_name}_DailyReturn")


# --- Load Trained Models ---
print("--- Loading Trained Sub-Agent Models (A2C, PPO only) ---")
try:
    model_a2c = A2C.load(A2C_MODEL_PATH)
    model_ppo = PPO.load(PPO_MODEL_PATH)
except Exception as e:
    print(f"Error loading models. Please ensure paths are correct and models are saved: {e}")
    exit()

# --- Simulate and add performance to df_wide ---
df_wide_with_performance = df_wide.copy()

# Simulate A2C performance
a2c_returns_series = simulate_agent_performance(model_a2c, Mult_Asset_portEnv, df_wide, "A2C", num_assets, features_list, window_size=5)
df_wide_with_performance = pd.concat([df_wide_with_performance, a2c_returns_series], axis=1)

# Simulate PPO performance
ppo_returns_series = simulate_agent_performance(model_ppo, Mult_Asset_portEnv, df_wide, "PPO", num_assets, features_list, window_size=5)
df_wide_with_performance = pd.concat([df_wide_with_performance, ppo_returns_series], axis=1)

# --- Save the updated df_wide with performance data ---
# Renamed output file to reflect A2C and PPO only
output_path_perf = '/home/micheal/Documents/Python_Library/RL_Optimization_Portfolio/data/processed/AP_MS_WIDE_with_A2C_PPO_Perf_daily.CSV'
df_wide_with_performance.to_csv(output_path_perf)
print(f"\nUpdated df_wide with A2C and PPO sub-agent performance saved to: {output_path_perf}")

print("\n--- Head of df_wide_with_performance (new columns) ---")
print(df_wide_with_performance.tail()) 
print(df_wide_with_performance.info())

--- Loading Trained Sub-Agent Models (A2C, PPO only) ---

--- Simulating A2C historical performance ---
DEBUG: Observation dtype at step 5: float32
DEBUG: Observation dtype at step 5: float32
DEBUG: Observation dtype at step 6: float32
DEBUG: Observation dtype at step 7: float32
DEBUG: Observation dtype at step 8: float32
DEBUG: Observation dtype at step 9: float32
DEBUG: Observation dtype at step 10: float32
DEBUG: Observation dtype at step 11: float32
DEBUG: Observation dtype at step 12: float32
DEBUG: Observation dtype at step 13: float32
DEBUG: Observation dtype at step 14: float32
DEBUG: Observation dtype at step 15: float32
DEBUG: Observation dtype at step 16: float32
DEBUG: Observation dtype at step 17: float32
DEBUG: Observation dtype at step 18: float32
DEBUG: Observation dtype at step 19: float32
DEBUG: Observation dtype at step 20: float32
DEBUG: Observation dtype at step 21: float32
DEBUG: Observation dtype at step 22: float32
DEBUG: Observation dtype at step 23: float32
DE

In [1]:
import pandas as pd
import numpy as np
import gym
from gym import spaces
from stable_baselines3 import DQN, A2C, PPO # Import DQN for meta-agent, A2C/PPO for sub-agents
from stable_baselines3.common.vec_env import DummyVecEnv

# Import your environments
from envs.portfolio_env import Mult_Asset_portEnv # Continuous for A2C/PPO sub-agents
from envs.meta_portfolio_env import MetaPortfolioEnv # Your new meta-environment

# --- IMPORTANT: ADJUST THESE PATHS TO YOUR SAVED MODELS AND DATA ---
A2C_MODEL_PATH = "/home/micheal/Documents/Python_Library/RL_Optimization_Portfolio/models/a2c_model"
PPO_MODEL_PATH = "/home/micheal/Documents/Python_Library/RL_Optimization_Portfolio/models/ppo_model"
META_DATA_PATH = '/home/micheal/Documents/Python_Library/RL_Optimization_Portfolio/data/processed/AP_MS_WIDE_with_A2C_PPO_Perf_daily.CSV'

# --- 1. Load the data with sub-agent performance ---
try:
    df_meta_data = pd.read_csv(META_DATA_PATH, parse_dates=['Date'], index_col='Date', engine='pyarrow', dtype_backend='pyarrow')
    # Defensive casting to float32 for all numeric columns
    for col in df_meta_data.columns:
        if pd.api.types.is_numeric_dtype(df_meta_data[col]):
            df_meta_data[col] = df_meta_data[col].astype(np.float32)
except FileNotFoundError:
    print(f"Error: The meta-agent data file '{META_DATA_PATH}' was not found.")
    print("Please ensure you have run the sub-agent performance simulation step correctly.")
    exit()

# --- 2. Load the trained sub-agent models ---
print("--- Loading Trained Sub-Agent Models for Meta-Agent Use ---")
try:
    model_a2c_sub = A2C.load(A2C_MODEL_PATH)
    model_ppo_sub = PPO.load(PPO_MODEL_PATH)
    sub_agent_models = {'A2C': model_a2c_sub, 'PPO': model_ppo_sub}
except Exception as e:
    print(f"Error loading sub-agent models for meta-agent. Please ensure paths are correct: {e}")
    exit()

# --- 3. Define common environment parameters for MetaPortfolioEnv ---
features_list_sub = ['Close', 'High', 'Low', 'Open', 'Volume', 'LogReturn', 'LogReturn_Z'] # Features sub-agents use
all_tickers = sorted(list(set([col.split('_')[0] for col in df_meta_data.columns if '_' in col])))
num_assets = len(all_tickers)
window_size_sub_agent = 5 # window_size used by the sub-agents for their observations
perf_window_size_meta = 20 # Lookback for meta-agent's performance features

# --- 4. Initialize MetaPortfolioEnv ---
print("\n--- Initializing Meta-Agent Portfolio Environment for DQN Training ---")

# The lambda function is crucial for DummyVecEnv
env_meta = DummyVecEnv([lambda: MetaPortfolioEnv(
    df_with_perf_data=df_meta_data,
    sub_agent_models=sub_agent_models,
    num_assets=num_assets,
    features_list=features_list_sub, # Pass features list used by sub-agents
    window_size=window_size_sub_agent, # Pass window size used by sub-agents
    perf_window_size=perf_window_size_meta
)])

# --- 5. Initialize and Train the Meta-Agent (DQN) ---
print("\n--- Initializing DQN Meta-Agent Model and Starting Training ---")
# 'MlpPolicy' for DQN works with discrete action spaces
# model_meta_dqn = DQN('MlpPolicy', env_meta, verbose=1)

# Start training for a specified number of timesteps
# Meta-agents might need more timesteps due to higher-level decision makbing
# model_meta_dqn.learn(total_timesteps=50000) # Increased timesteps for meta-agent example

# print("\n--- DQN Meta-Agent Model Training Finished ---")
print("You can now evaluate the trained meta-agent model or save it for later use.")

2025-07-21 21:00:33.735550: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1753128033.785306   59421 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1753128033.799120   59421 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-07-21 21:00:33.843010: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2, in other operations, rebuild TensorFlow with the appropriate compiler flags.


--- Loading Trained Sub-Agent Models for Meta-Agent Use ---

--- Initializing Meta-Agent Portfolio Environment for DQN Training ---

--- Initializing DQN Meta-Agent Model and Starting Training ---
You can now evaluate the trained meta-agent model or save it for later use.


/home/micheal/anaconda3/lib/python3.12/site-packages/stable_baselines3/common/vec_env/patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


In [3]:
import pandas as pd
import numpy as np
import gym
from gym import spaces
from stable_baselines3 import PPO, A2C # Import PPO for meta-agent, A2C for sub-agents
from stable_baselines3.common.vec_env import DummyVecEnv

# --- Start: Definition of Mult_Asset_portEnv (your continuous environment) ---
# This class needs to be defined for MetaPortfolioEnv to use.
# Ensure this matches your envs/portfolio_env.py
class Mult_Asset_portEnv(gym.Env):
    def __init__(self, df, num_assets, features_list, window_size=5, initial_balance=1000, transaction_cost_rate=0.001):
        super(Mult_Asset_portEnv, self).__init__()
        
        self.df = df.copy()
        self.num_assets = num_assets
        self.features_list = features_list
        self.num_features_per_asset = len(features_list)
        self.window_size = window_size
        self.initial_balance = initial_balance
        self.transaction_cost_rate = transaction_cost_rate
        
        self.action_space = spaces.Box(low=0.0, high=1.0,
                                       shape=(self.num_assets + 1,), 
                                       dtype=np.float32)
        
        num_portfolio_features = self.num_assets + 1
        observation_row_size = self.num_assets * self.num_features_per_asset + num_portfolio_features
        
        self.observation_space = spaces.Box(
            low=-np.inf, high=np.inf, shape=(window_size, observation_row_size), 
            dtype=np.float32
        )
        
        if len(self.df) < self.window_size + 1:
            raise ValueError(f"DataFrame too short for window_size {self.window_size}. "
                             f"Requires at least {self.window_size + 1} rows, but has {len(self.df)}.")
        
        self.reset()
        
    def reset(self):
        self.current_step = self.window_size
        self.balance = self.initial_balance
        self.cash = self.initial_balance
        self.portfolio_weights = np.array([0.0] * self.num_assets + [1.0], dtype=np.float32)
        self.assets_shares = np.array([0.0] * self.num_assets, dtype=np.float32)
        self.history = []
        self.peak_portfolio_value = self.initial_balance

        return self._get_observation()

    def _get_observation(self):
        start_idx = self.current_step - self.window_size
        end_idx = self.current_step

        # Filter only actual asset tickers (not agent prefixes like A2C_, PPO_, etc.)
        valid_tickers = sorted(list(set([
            col.split('_')[0] for col in self.df.columns
            if '_' in col and not col.startswith(('A2C_', 'PPO_', 'DQN_'))
        ])))

        all_feature_columns = [
            f"{ticker}_{feature}"
            for ticker in valid_tickers
            for feature in self.features_list
        ]
        
        missing_cols = [col for col in all_feature_columns if col not in self.df.columns]
        if missing_cols:
            raise ValueError(f"Missing expected feature columns in df_wide: {missing_cols}. "
                             f"Please check features_list and df_wide structure.")

        features_window_data = self.df.loc[self.df.index[start_idx:end_idx], all_feature_columns].values.astype(np.float32)

        current_portfolio_state_repeated = np.tile(self.portfolio_weights, (self.window_size, 1))

        observation = np.concatenate([features_window_data, current_portfolio_state_repeated], axis=1)
        
        return observation.astype(np.float32)

    def step(self, action):
        action = action / np.sum(action) 
        self.portfolio_weights = action

        current_data_row = self.df.iloc[self.current_step]
        
        asset_tickers_in_order = sorted(list(set([
            col.split('_')[0] for col in self.df.columns
            if '_' in col and not col.startswith(('A2C_', 'PPO_', 'DQN_'))
        ])))
        
        current_close_prices = np.array([current_data_row[f"{ticker}_Close"] for ticker in asset_tickers_in_order], dtype=np.float32)
        current_log_returns = np.array([current_data_row[f"{ticker}_LogReturn"] for ticker in asset_tickers_in_order], dtype=np.float32)

        current_close_prices = np.maximum(current_close_prices, 1e-6)

        current_assets_market_value = np.sum(self.assets_shares * current_close_prices)
        total_portfolio_value_before_trades = self.cash + current_assets_market_value
        
        if total_portfolio_value_before_trades <= 0:
            reward = -100
            done = True
            return self._get_observation(), reward, done, {}

        target_assets_value = action[:-1] * total_portfolio_value_before_trades
        target_cash_value = action[-1] * total_portfolio_value_before_trades

        target_assets_shares = target_assets_value / current_close_prices
        
        shares_to_buy_sell = target_assets_shares - self.assets_shares

        cash_flow_from_trades = np.sum(shares_to_buy_sell * current_close_prices)
        transaction_costs = np.sum(np.abs(shares_to_buy_sell * current_close_prices)) * self.transaction_cost_rate

        new_cash = self.cash - cash_flow_from_trades - transaction_costs
        new_assets_shares = self.assets_shares + shares_to_buy_sell

        if new_cash < 0:
            reward = -100
            done = True
            return self._get_observation(), reward, done, {}
        
        self.cash = new_cash
        self.assets_shares = new_assets_shares

        new_assets_market_value = np.sum(self.assets_shares * current_close_prices)
        self.balance = self.cash + new_assets_market_value

        epsilon = 1e-9 
        portfolio_daily_return = (self.balance - total_portfolio_value_before_trades) / (total_portfolio_value_before_trades + epsilon)
        
        max_return_clip = 0.1 
        min_return_clip = -0.1 
        portfolio_daily_return = np.clip(portfolio_daily_return, min_return_clip, max_return_clip)

        risk_penalty = 0.0001 
        reward = portfolio_daily_return - risk_penalty
        
        self.history.append(self.balance)
        self.peak_portfolio_value = max(self.peak_portfolio_value, self.balance) 

        self.current_step += 1
        done = self.current_step >= len(self.df) or self.balance <= 0 or self.balance < (self.initial_balance * 0.1) 
        
        info = {}
        if done:
            if self.balance <= 0: info['reason'] = 'bankrupt'
            elif self.balance < (self.initial_balance * 0.1): info['reason'] = 'low_balance'
            elif self.current_step >= len(self.df): info['reason'] = 'end_of_data'

        return self._get_observation(), reward, done, info

    def render(self, mode='human'):
        if mode == 'human':
            print(f"Step {self.current_step}, Total Value: {self.balance:.2f}")

# --- End: Definition of Mult_Asset_portEnv ---


# --- Start: Definition of MetaPortfolioEnv ---
# Ensure this matches your envs/meta_portfolio_env.py
class MetaPortfolioEnv(gym.Env):
    def __init__(self, df_with_perf_data, sub_agent_models, num_assets, features_list, window_size=5, initial_balance=1000, transaction_cost_rate=0.001, perf_window_size=20):
        super(MetaPortfolioEnv, self).__init__()

        self.df = df_with_perf_data.copy()
        self.sub_agent_models = sub_agent_models 
        self.num_assets = num_assets
        self.features_list = features_list 
        self.window_size = window_size 
        self.initial_balance = initial_balance
        self.transaction_cost_rate = transaction_cost_rate
        self.perf_window_size = perf_window_size 

        self.num_sub_agents = len(self.sub_agent_models)
        self.action_space = spaces.Discrete(self.num_sub_agents)

        num_market_features = 2 
        num_portfolio_state_features = 1 + 1 + (self.num_assets + 1) 
        num_sub_agent_perf_features = self.num_sub_agents * 2 

        num_meta_features = num_market_features + num_portfolio_state_features + num_sub_agent_perf_features
        
        self.observation_space = spaces.Box(
            low=-np.inf, high=np.inf, shape=(num_meta_features,), dtype=np.float32 
        )

        if len(self.df) < self.window_size + self.perf_window_size + 1:
            raise ValueError(f"DataFrame too short for meta-agent environment. Requires at least "
                             f"{self.window_size + self.perf_window_size + 1} rows, but has {len(self.df)}.")

        self.reset()

    def reset(self):
        self.current_step = self.window_size + self.perf_window_size
        self.balance = self.initial_balance
        self.cash = self.initial_balance
        self.portfolio_weights = np.array([0.0] * self.num_assets + [1.0], dtype=np.float32) 
        self.assets_shares = np.array([0.0] * self.num_assets, dtype=np.float32)

        self.history = []
        self.peak_portfolio_value = self.initial_balance
        
        return self._get_observation()

    def _get_observation(self):
        if self.current_step < self.window_size + self.perf_window_size:
            return np.zeros(self.observation_space.shape, dtype=np.float32)

        market_obs_start_idx = self.current_step - self.window_size
        market_obs_end_idx = self.current_step
        
        market_log_return_z_cols = [
            col for col in self.df.columns 
            if col.endswith('_LogReturn_Z') and not col.startswith(('A2C_', 'PPO_', 'DQN_'))
        ]
        
        if not market_log_return_z_cols:
            raise RuntimeError("No 'TICKER_LogReturn_Z' columns found for market observation in df_meta_data.")

        market_window_data = self.df.loc[self.df.index[market_obs_start_idx:market_obs_end_idx], market_log_return_z_cols].values
        
        mean_log_return_z = np.mean(market_window_data)
        std_log_return_z = np.std(market_window_data)

        normalized_balance = self.balance / self.initial_balance
        cash_percentage = self.cash / self.balance if self.balance > 0 else 0.0

        perf_data_start_idx = self.current_step - self.perf_window_size
        perf_data_end_idx = self.current_step

        a2c_perf = self.df.loc[self.df.index[perf_data_start_idx:perf_data_end_idx], 'A2C_DailyReturn'].values
        ppo_perf = self.df.loc[self.df.index[perf_data_start_idx:perf_data_end_idx], 'PPO_DailyReturn'].values
        
        a2c_perf = np.nan_to_num(a2c_perf, nan=0.0)
        ppo_perf = np.nan_to_num(ppo_perf, nan=0.0)

        a2c_mean_perf = np.mean(a2c_perf)
        a2c_std_perf = np.std(a2c_perf)
        ppo_mean_perf = np.mean(ppo_perf)
        ppo_std_perf = np.std(ppo_perf)

        meta_observation = np.concatenate([
            np.array([mean_log_return_z, std_log_return_z]), 
            np.array([normalized_balance, cash_percentage]), 
            self.portfolio_weights,                          
            np.array([a2c_mean_perf, a2c_std_perf, ppo_mean_perf, ppo_std_perf]) 
        ])

        if meta_observation.shape != self.observation_space.shape:
            raise RuntimeError(f"Meta-observation shape mismatch: Expected {self.observation_space.shape}, got {meta_observation.shape}. "
                               f"Calculated components: Market {np.array([mean_log_return_z, std_log_return_z]).shape}, "
                               f"Portfolio State {np.array([normalized_balance, cash_percentage]).shape}, "
                               f"Weights {self.portfolio_weights.shape}, "
                               f"Sub-agent Perf {np.array([a2c_mean_perf, a2c_std_perf, ppo_mean_perf, ppo_std_perf]).shape}")

        return meta_observation.astype(np.float32)

    def step(self, action):
        done = False
        info = {}

        chosen_agent_name = None
        chosen_sub_model = None
        
        if action == 0:
            chosen_agent_name = 'A2C'
            chosen_sub_model = self.sub_agent_models['A2C']
        elif action == 1:
            chosen_agent_name = 'PPO'
            chosen_sub_model = self.sub_agent_models['PPO']
        else:
            raise ValueError(f"Invalid meta-agent action: {action}. Must be 0 (A2C) or 1 (PPO).")

        temp_env = Mult_Asset_portEnv(df=self.df.copy(), num_assets=self.num_assets, features_list=self.features_list, window_size=self.window_size, initial_balance=self.initial_balance, transaction_cost_rate=self.transaction_cost_rate)
        temp_env.current_step = self.current_step 
        temp_obs = temp_env._get_observation() 

        sub_agent_action, _states = chosen_sub_model.predict(temp_obs, deterministic=True)
        
        sub_agent_action = sub_agent_action / np.sum(sub_agent_action) 
        self.portfolio_weights = sub_agent_action 

        current_data_row = self.df.iloc[self.current_step]
        asset_tickers_in_order = sorted(list(set([
            col.split('_')[0] for col in self.df.columns
            if '_' in col and not col.startswith(('A2C_', 'PPO_', 'DQN_'))
        ])))
        
        current_close_prices = np.array([current_data_row[f"{ticker}_Close"] for ticker in asset_tickers_in_order], dtype=np.float32)
        current_log_returns = np.array([current_data_row[f"{ticker}_LogReturn"] for ticker in asset_tickers_in_order], dtype=np.float32)

        current_close_prices = np.maximum(current_close_prices, 1e-6)

        current_assets_market_value = np.sum(self.assets_shares * current_close_prices)
        total_portfolio_value_before_trades = self.cash + current_assets_market_value
        
        if total_portfolio_value_before_trades <= 0:
            reward = -100
            done = True
            info['reason'] = 'bankrupt'
            return self._get_observation(), reward, done, info

        target_assets_value = sub_agent_action[:-1] * total_portfolio_value_before_trades
        target_cash_value = sub_agent_action[-1] * total_portfolio_value_before_trades

        target_assets_shares = target_assets_value / current_close_prices
        shares_to_buy_sell = target_assets_shares - self.assets_shares

        cash_flow_from_trades = np.sum(shares_to_buy_sell * current_close_prices)
        transaction_costs = np.sum(np.abs(shares_to_buy_sell * current_close_prices)) * self.transaction_cost_rate

        new_cash = self.cash - cash_flow_from_trades - transaction_costs
        new_assets_shares = self.assets_shares + shares_to_buy_sell

        if new_cash < 0:
            reward = -100
            done = True
            info['reason'] = 'meta_cash_negative'
            return self._get_observation(), reward, done, info
        
        self.cash = new_cash
        self.assets_shares = new_assets_shares

        new_assets_market_value = np.sum(self.assets_shares * current_close_prices)
        self.balance = self.cash + new_assets_market_value

        epsilon_balance = 1e-9 
        portfolio_daily_return = (self.balance - total_portfolio_value_before_trades) / (total_portfolio_value_before_trades + epsilon_balance)
        
        max_return_clip = 0.1 
        min_return_clip = -0.1 
        portfolio_daily_return = np.clip(portfolio_daily_return, min_return_clip, max_return_clip)

        risk_penalty = 0.0001 
        reward = portfolio_daily_return - risk_penalty 

        self.history.append(self.balance)
        self.peak_portfolio_value = max(self.peak_portfolio_value, self.balance) 

        self.current_step += 1
        done = self.current_step >= len(self.df) or self.balance <= 0 or self.balance < (self.initial_balance * 0.1) 
        
        if done:
            if self.balance <= 0: info['reason'] = 'bankrupt'
            elif self.balance < (self.initial_balance * 0.1): info['reason'] = 'low_balance'
            elif self.current_step >= len(self.df): info['reason'] = 'end_of_data'

        info['chosen_agent'] = chosen_agent_name 
        self.chosen_agent_name = chosen_agent_name 
        
        return self._get_observation(), reward, done, info
        
    def render(self, mode='human'):
        if mode == 'human':
            print(f"Step {self.current_step}, Total Port Value: {self.balance:.2f}, Cash: {self.cash:.2f}, "
                  f"Weights: {[f'{w:.2f}' for w in self.portfolio_weights]}, Chosen Agent: {self.chosen_agent_name if hasattr(self, 'chosen_agent_name') else 'N/A'}")

# --- End: Definition of MetaPortfolioEnv ---


# --- Script for training the PPO Meta-Agent ---

# --- IMPORTANT: ADJUST THESE PATHS TO YOUR SAVED MODELS AND DATA ---
A2C_MODEL_PATH = "/home/micheal/Documents/Python_Library/RL_Optimization_Portfolio/models/a2c_model"
PPO_MODEL_PATH = "/home/micheal/Documents/Python_Library/RL_Optimization_Portfolio/models/ppo_model"
META_DATA_PATH = '/home/micheal/Documents/Python_Library/RL_Optimization_Portfolio/data/processed/AP_MS_WIDE_with_A2C_PPO_Perf_daily.CSV'

# --- 1. Load the data with sub-agent performance ---
try:
    df_meta_data = pd.read_csv(META_DATA_PATH, parse_dates=['Date'], index_col='Date', engine='pyarrow', dtype_backend='pyarrow')
    for col in df_meta_data.columns:
        if pd.api.types.is_numeric_dtype(df_meta_data[col]):
            df_meta_data[col] = df_meta_data[col].astype(np.float32)
except FileNotFoundError:
    print(f"Error: The meta-agent data file '{META_DATA_PATH}' was not found.")
    print("Please ensure you have run the the sub-agent performance simulation step correctly.")
    exit()

# --- 2. Load the trained sub-agent models ---
print("--- Loading Trained Sub-Agent Models for Meta-Agent Use ---")
try:
    model_a2c_sub = A2C.load(A2C_MODEL_PATH)
    model_ppo_sub = PPO.load(PPO_MODEL_PATH)
    sub_agent_models = {'A2C': model_a2c_sub, 'PPO': model_ppo_sub}
except Exception as e:
    print(f"Error loading sub-agent models for meta-agent. Please ensure paths are correct: {e}")
    exit()

# --- 3. Define common environment parameters for MetaPortfolioEnv ---
features_list_sub = ['Close', 'High', 'Low', 'Open', 'Volume', 'LogReturn', 'LogReturn_Z'] 

all_tickers = sorted(list(set([col.split('_')[0] for col in df_meta_data.columns 
                               if '_' in col and not col.startswith(('A2C_', 'PPO_', 'DQN_'))])))
num_assets = len(all_tickers)

window_size_sub_agent = 5 
perf_window_size_meta = 20 

# --- 4. Initialize MetaPortfolioEnv ---
print("\n--- Initializing Meta-Agent Portfolio Environment for PPO Training ---")

env_meta = DummyVecEnv([lambda: MetaPortfolioEnv(
    df_with_perf_data=df_meta_data,
    sub_agent_models=sub_agent_models,
    num_assets=num_assets,
    features_list=features_list_sub, 
    window_size=window_size_sub_agent, 
    perf_window_size=perf_window_size_meta
)])

# --- 5. Initialize and Train the Meta-Agent (PPO) ---
print("\n--- Initializing PPO Meta-Agent Model and Starting Training ---")
model_meta_ppo = PPO('MlpPolicy', env_meta, verbose=1)

model_meta_ppo.learn(total_timesteps=50000) 

print("\n--- PPO Meta-Agent Model Training Finished ---")
print("You can now evaluate the trained meta-agent model or save it for later use.")

--- Loading Trained Sub-Agent Models for Meta-Agent Use ---

--- Initializing Meta-Agent Portfolio Environment for PPO Training ---

--- Initializing PPO Meta-Agent Model and Starting Training ---
Using cpu device


/home/micheal/anaconda3/lib/python3.12/site-packages/stable_baselines3/common/vec_env/patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


-----------------------------
| time/              |      |
|    fps             | 68   |
|    iterations      | 1    |
|    time_elapsed    | 29   |
|    total_timesteps | 2048 |
-----------------------------
------------------------------------------
| time/                   |              |
|    fps                  | 67           |
|    iterations           | 2            |
|    time_elapsed         | 61           |
|    total_timesteps      | 4096         |
| train/                  |              |
|    approx_kl            | 0.0020040064 |
|    clip_fraction        | 0.00479      |
|    clip_range           | 0.2          |
|    entropy_loss         | -0.692       |
|    explained_variance   | -2.38        |
|    learning_rate        | 0.0003       |
|    loss                 | 0.00937      |
|    n_updates            | 10           |
|    policy_gradient_loss | -0.00114     |
|    value_loss           | 0.0015       |
------------------------------------------
----------------